## Main Jupyter Notebooks for implementing the German Traffic Sign Classification CNN

### Read in Data

In [1]:
import matplotlib.pyplot as plt
import csv
from PIL import Image
import numpy as np

In [2]:
def readTrafficSigns(rootpath):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        next(gtReader)  # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            images.append(plt.imread(prefix + row[0])) # the 1th column is the filename
            print("Error reading image: ", prefix + row[0])
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
    return images, labels

In [3]:
img, lables = readTrafficSigns('/Users/linuspfeifer/Documents/Uni/5. Semester/XAI/Abgabe/GTSRB/Final_Training/Images')

### Train the Model

In [1]:
import torch